# Imports

In [1]:
from IPython.display import display as d
from omegaconf import OmegaConf

import torch
from torch import nn

import matplotlib.pyplot as plt

import pickle

from typing import Optional, Tuple, List, Union, Dict

import pandas as pd
import numpy as np

import lz4
import lz4.frame

from tqdm import tqdm

import sys

# Serialize and compress data

In [2]:
sample = pd.read_parquet("./data/credits-history/train_data/train_data_0.pq")

In [16]:
sample_targets = pd.read_parquet("./data/credits-history/train_targets")

In [36]:
with open("./data/credits-history/train_targets/targets_dict.pickle", "wb") as f:
    pickle.dump(dict(zip(sample_targets["id"], sample_targets["flag"])), f)

In [18]:
sample_targets = pd.read_pickle("./data/credits-history/targets/targets_dict.pickle")

In [3]:
d(sample.columns)

Index(['id', 'rn', 'pre_since_opened', 'pre_since_confirmed', 'pre_pterm',
       'pre_fterm', 'pre_till_pclose', 'pre_till_fclose',
       'pre_loans_credit_limit', 'pre_loans_next_pay_summ',
       'pre_loans_outstanding', 'pre_loans_total_overdue',
       'pre_loans_max_overdue_sum', 'pre_loans_credit_cost_rate', 'pre_loans5',
       'pre_loans530', 'pre_loans3060', 'pre_loans6090', 'pre_loans90',
       'is_zero_loans5', 'is_zero_loans530', 'is_zero_loans3060',
       'is_zero_loans6090', 'is_zero_loans90', 'pre_util', 'pre_over2limit',
       'pre_maxover2limit', 'is_zero_util', 'is_zero_over2limit',
       'is_zero_maxover2limit', 'enc_paym_0', 'enc_paym_1', 'enc_paym_2',
       'enc_paym_3', 'enc_paym_4', 'enc_paym_5', 'enc_paym_6', 'enc_paym_7',
       'enc_paym_8', 'enc_paym_9', 'enc_paym_10', 'enc_paym_11', 'enc_paym_12',
       'enc_paym_13', 'enc_paym_14', 'enc_paym_15', 'enc_paym_16',
       'enc_paym_17', 'enc_paym_18', 'enc_paym_19', 'enc_paym_20',
       'enc_paym_21', 

In [34]:
d(sample["id"].drop_duplicates())

0               0
10              1
24              2
27              3
42              4
            ...  
1974673    249995
1974686    249996
1974709    249997
1974716    249998
1974721    249999
Name: id, Length: 250000, dtype: int64

In [4]:
d(sample)

,id,rn,pre_since_opened,pre_since_confirmed,pre_pterm,pre_fterm,pre_till_pclose,pre_till_fclose,pre_loans_credit_limit,pre_loans_next_pay_summ,...,enc_paym_21,enc_paym_22,enc_paym_23,enc_paym_24,enc_loans_account_holder_type,enc_loans_credit_status,enc_loans_credit_type,enc_loans_account_cur,pclose_flag,fclose_flag
0,0,1,18,9,2,3,16,10,11,3,...,3,3,3,4,1,3,4,1,0,0
1,0,2,18,9,14,14,12,12,0,3,...,0,0,0,4,1,3,4,1,0,0
2,0,3,18,9,4,8,1,11,11,0,...,0,0,0,4,1,2,3,1,1,1
3,0,4,4,1,9,12,16,7,12,2,...,3,3,3,4,1,3,1,1,0,0
4,0,5,5,12,15,2,11,12,10,2,...,3,3,3,4,1,3,4,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1974719,249998,4,1,9,14,8,10,11,8,4,...,3,3,3,4,1,2,3,1,0,1
1974720,249998,5,7,9,4,8,1,11,19,1,...,3,3,3,4,1,2,4,1,1,1
1974721,249999,1,9,0,10,8,10,11,16,2,...,3,3,3,4,1,2,3,1,0,1
1974722,249999,2,9,16,10,13,10,4,12,2,...,3,3,3,4,1,2,3,1,0,0


In [6]:
d(sample[sample.columns[30:55]])

,enc_paym_0,enc_paym_1,enc_paym_2,enc_paym_3,enc_paym_4,enc_paym_5,enc_paym_6,enc_paym_7,enc_paym_8,enc_paym_9,...,enc_paym_16,enc_paym_17,enc_paym_18,enc_paym_19,enc_paym_20,enc_paym_21,enc_paym_22,enc_paym_23,enc_paym_24,enc_loans_account_holder_type
0,0,0,3,3,3,3,3,3,3,3,...,3,3,3,3,4,3,3,3,4,1
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,4,1
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,4,1
3,1,0,0,0,0,0,0,0,0,0,...,3,3,3,3,4,3,3,3,4,1
4,0,0,0,0,0,0,0,3,3,3,...,3,3,3,3,4,3,3,3,4,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1974719,0,0,0,0,0,0,0,0,0,0,...,3,3,3,3,4,3,3,3,4,1
1974720,0,0,0,0,0,3,3,3,3,3,...,3,3,3,3,4,3,3,3,4,1
1974721,0,0,0,0,0,0,0,0,3,3,...,3,3,3,3,4,3,3,3,4,1
1974722,0,0,0,0,0,0,0,0,0,3,...,3,3,3,3,4,3,3,3,4,1


In [7]:
# sample["enc_paym_max"] = sample[sample.columns[30:55]].max(axis=1)

# sample["enc_paym_avg"] = sample[sample.columns[30:55]].mean(axis=1)
# sample["enc_paym_sin"] = np.sin(sample["enc_paym_avg"])
# sample["enc_paym_cos"] = np.cos(sample["enc_paym_avg"])

sample["enc_paym_1_to_8_max"] = sample[sample.columns[31:39]].max(axis=1)
sample["enc_paym_1_to_8_sum"] = sample[sample.columns[31:39]].sum(axis=1)

sample["enc_paym_9_to_16_max"] = sample[sample.columns[39:47]].max(axis=1)
sample["enc_paym_9_to_16_sum"] = sample[sample.columns[39:47]].sum(axis=1)

sample["enc_paym_17_to_24_max"] = sample[sample.columns[47:55]].max(axis=1)
sample["enc_paym_17_to_24_sum"] = sample[sample.columns[47:55]].sum(axis=1)

sample.drop(sample.columns[30:55], axis=1, inplace=True)

In [8]:
sample["pre_loans5_masked"] = sample["is_zero_loans5"].map(lambda x: 1 - x) * sample["pre_loans5"]
sample["pre_loans530_masked"] = sample["is_zero_loans530"].map(lambda x: 1 - x) * sample["pre_loans530"]
sample["pre_loans3060_masked"] = sample["is_zero_loans3060"].map(lambda x: 1 - x) * sample["pre_loans3060"]
sample["pre_loans6090_masked"] = sample["is_zero_loans6090"].map(lambda x: 1 - x) * sample["pre_loans6090"]
sample["pre_loans90_masked"] = sample["is_zero_loans90"].map(lambda x: 1 - x) * sample["pre_loans90"]

sample.drop(
    [
        "is_zero_loans5", "is_zero_loans530", "is_zero_loans3060", "is_zero_loans6090", "is_zero_loans90",
        "pre_loans5", "pre_loans530", "pre_loans3060", "pre_loans6090", "pre_loans90"
    ], axis=1, inplace=True
)

In [9]:
sample["pre_loans5_masked"] = sample["pre_loans5_masked"].replace({7: 6, 13: 6, 16: 6})
sample["pre_loans530_masked"] = sample["pre_loans530_masked"].replace({7: 16, 11: 16, 10: 16, 1: 16, 14: 16, 4: 16})
sample["pre_loans3060_masked"] = sample["pre_loans3060_masked"].replace({9: 5, 7: 5, 2: 5})
sample["pre_loans6090_masked"] = sample["pre_loans6090_masked"].replace({2: 1})
sample["pre_loans90_masked"] = sample["pre_loans90_masked"].replace({19: 8, 13: 8})

In [54]:
# sample["pre_loans5_masked"] = sample["pre_loans5_masked"].replace(
#     {old_value: new_value for new_value, old_value in enumerate(sample["pre_loans5_masked"].value_counts().keys())}
# )
# sample["pre_loans530_masked"] = sample["pre_loans530_masked"].replace(
#     {old_value: new_value for new_value, old_value in enumerate(sample["pre_loans530_masked"].value_counts().keys())}
# )
# sample["pre_loans3060_masked"] = sample["pre_loans3060_masked"].replace(
#     {old_value: new_value for new_value, old_value in enumerate(sample["pre_loans3060_masked"].value_counts().keys())}
# )
# sample["pre_loans6090_masked"] = sample["pre_loans6090_masked"].replace(
#     {old_value: new_value for new_value, old_value in enumerate(sample["pre_loans6090_masked"].value_counts().keys())}
# )
# sample["pre_loans90_masked"] = sample["pre_loans90_masked"].replace(
#     {old_value: new_value for new_value, old_value in enumerate(sample["pre_loans90_masked"].value_counts().keys())}
# )

In [10]:
sample["pre_util_masked"] = sample["is_zero_util"].map(lambda x: 1 - x) * sample["pre_util"]
sample["pre_over2limit_masked"] = sample["is_zero_over2limit"].map(lambda x: 1 - x) * sample["pre_over2limit"]
sample["pre_maxover2limit_masked"] = sample["is_zero_maxover2limit"].map(lambda x: 1 - x) * sample["pre_maxover2limit"]

sample.drop(
    [
        "is_zero_util", "is_zero_over2limit", "is_zero_maxover2limit",
        "pre_util", "pre_over2limit", "pre_maxover2limit"
    ], axis=1, inplace=True
)

In [11]:
sample.drop("rn", axis=1, inplace=True)

In [12]:
d(sample)

,id,pre_since_opened,pre_since_confirmed,pre_pterm,pre_fterm,pre_till_pclose,pre_till_fclose,pre_loans_credit_limit,pre_loans_next_pay_summ,pre_loans_outstanding,...,enc_paym_sin,enc_paym_cos,pre_loans5_masked,pre_loans530_masked,pre_loans3060_masked,pre_loans6090_masked,pre_loans90_masked,pre_util_masked,pre_over2limit_masked,pre_maxover2limit_masked
0,0,18,9,2,3,16,10,11,3,3,...,0.258619,-0.965979,0,0,0,0,0,0,0,0
1,0,18,9,14,14,12,12,0,3,3,...,0.237703,0.971338,0,0,0,0,0,0,0,0
2,0,18,9,4,8,1,11,11,0,5,...,0.237703,0.971338,0,0,0,0,0,15,0,0
3,0,4,1,9,12,16,7,12,2,3,...,0.988890,-0.148651,6,0,0,0,0,0,0,0
4,0,5,12,15,2,11,12,10,2,3,...,0.758881,-0.651230,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1974719,249998,1,9,14,8,10,11,8,4,2,...,0.999942,0.010796,0,0,0,0,0,11,0,0
1974720,249998,7,9,4,8,1,11,19,1,4,...,0.582331,-0.812952,0,0,0,0,0,7,0,0
1974721,249999,9,0,10,8,10,11,16,2,3,...,0.831383,-0.555699,0,0,0,0,0,0,0,0
1974722,249999,9,16,10,13,10,4,12,2,3,...,0.891929,-0.452176,0,0,0,0,0,0,0,0


In [75]:
features_dict = {col: max(sample[col]) + 1 for col in sample.columns[2:] if np.unique(sample[col]).__len__() > 1}

In [76]:
d(features_dict)

{'pre_since_confirmed': 18,
 'pre_pterm': 18,
 'pre_fterm': 17,
 'pre_till_pclose': 17,
 'pre_till_fclose': 16,
 'pre_loans_credit_limit': 20,
 'pre_loans_next_pay_summ': 7,
 'pre_loans_outstanding': 6,
 'pre_loans_max_overdue_sum': 4,
 'pre_loans_credit_cost_rate': 14,
 'enc_paym_0': 4,
 'enc_loans_account_holder_type': 7,
 'enc_loans_credit_status': 7,
 'enc_loans_credit_type': 6,
 'enc_loans_account_cur': 4,
 'pclose_flag': 2,
 'fclose_flag': 2,
 'enc_paym_1_to_8_max': 4,
 'enc_paym_1_to_8_sum': 25,
 'enc_paym_9_to_16_max': 5,
 'enc_paym_9_to_16_sum': 26,
 'enc_paym_17_to_24_max': 5,
 'enc_paym_17_to_24_sum': 27,
 'pre_loans5_masked': 7,
 'pre_loans530_masked': 19,
 'pre_loans3060_masked': 9,
 'pre_loans6090_masked': 5,
 'pre_loans90_masked': 15,
 'pre_util_masked': 20,
 'pre_over2limit_masked': 20,
 'pre_maxover2limit_masked': 20}

In [77]:
OmegaConf.save(OmegaConf.create(features_dict), "configs/data/features/features_credits_aggregated.yaml")

In [78]:
features_dict = OmegaConf.load("configs/data/features/features_credits_aggregated.yaml")

In [ ]:
def serialize_and_compress(item: Dict):
    serialized_item = pickle.dumps(item)
    compressed = lz4.frame.compress(serialized_item)
    return compressed

def deserialize_and_decompress(item):
    decompressed = lz4.frame.decompress(item)
    deserialized = pickle.loads(decompressed)
    return deserialized


class DiskFile:
    def __init__(self, data_path: str, indexes_path: str):
        self.data_path = data_path
        self.indexes_path = indexes_path

        with open(self.indexes_path, "rb") as indexes:
            self.indexes = deserialize_and_decompress(indexes.read())

        self.data = open(data_path, "rb")
            

    def __getitem__(self, idx):
        memory_position, sizeof = self.indexes[idx]
        self.data.seek(memory_position)
        
        item = deserialize_and_decompress(self.data.read(sizeof))
        
        return item
    

In [79]:
def preprocess_part(left_id: int, right_id: int):
    sample_batch = sample.query(f"id >= {left_id} and id < {right_id}")

    sample_batch_collected = sample_batch.groupby("id").agg(pd.Series.tolist)
    sample_batch_collected["length"] = sample_batch_collected[sample_batch_collected.columns[0]].map(lambda x: len(x))

    result = list()
    for idx in sample_batch_collected.index:
        result.append({"id": idx, "features_array": serialize_and_compress(dict(sample_batch_collected.loc[idx]))})

    return result

In [89]:
batch_size = 10_000
n_batches = max(sample["id"]) // 10_000 + 1

path_to_dir = "./data/credits-history/serialized/ser_full_0"

file_indices = dict()

with open(path_to_dir + "/data.txt", "wb") as output:
    for i in tqdm(range(n_batches), ncols=80):
        preprocessed = preprocess_part(batch_size * i, batch_size * i + batch_size)

        for record in preprocessed:
            item = record["features_array"]
            file_indices[str(record["id"])] = (output.tell(), sys.getsizeof(item))
            output.write(item)


with open(path_to_dir + "/index.txt", "wb") as output:
    output.write(serialize_and_compress(file_indices))

100%|███████████████████████████████████████████| 25/25 [04:05<00:00,  9.81s/it]


# Dataloader sample & model step

In [2]:
from src.data.components.dataset import CustomDataset
from src.data.components.targets_indexes_reader import TargetsReader, IndexesReader
from src.data.components.data_reader import DataReader
from src.data.components.collate import BaseCollator, ModelBatch

from src.models.tabnn import TabularRNNLitModule

In [3]:
features_dict = OmegaConf.load("configs/data/features/features_credits_aggregated.yaml")

In [4]:
data_reader = DataReader(data_path="./data/credits-history/serialized/ser_full_0")
data_reader.setup()

In [5]:
boosting_dataset = CustomDataset(
    data=data_reader, 
    targets=TargetsReader("./data/credits-history/targets/targets_dict.pickle").targets,
    indexes=IndexesReader(train_path="./data/credits-history/indexes/ser_full_0_indexes/train_indexes.pickle").train_indexes,
    categorical_features=features_dict
)

In [6]:
d(boosting_dataset[0])

{'categorical': tensor([[ 7, 13,  2,  4,  1, 12,  2,  3,  2,  4,  0,  1,  3,  4,  1,  0,  0,  0,
           0,  4, 22,  4, 26,  0,  0,  0,  0,  0,  0,  0,  0],
         [ 7,  8, 11,  4,  1,  6,  2,  3,  2,  4,  0,  1,  3,  4,  1,  0,  0,  3,
          18,  4, 25,  4, 26,  0,  0,  0,  0,  0,  0,  0,  0],
         [ 7,  7,  6,  9,  1, 19,  2,  3,  2,  4,  0,  1,  3,  4,  1,  0,  0,  3,
           9,  4, 25,  4, 26,  0,  0,  0,  0,  0,  0,  0,  0],
         [17,  8,  6,  9,  1, 19,  2,  3,  2,  4,  0,  1,  3,  1,  1,  0,  0,  3,
           9,  4, 25,  4, 26,  0,  0,  0,  0,  0,  0,  0,  0],
         [ 9,  9, 11,  6, 13,  1,  4,  3,  2,  5,  0,  1,  3,  4,  1,  0,  0,  3,
          12,  4, 25,  4, 26,  0,  0,  0,  0,  0,  0,  0,  0],
         [ 9,  4, 10,  1,  5, 17,  4,  3,  2,  7,  0,  1,  3,  3,  1,  1,  0,  0,
           0,  1,  1,  4, 11,  0,  0,  0,  0,  0,  0,  0,  0],
         [ 9,  4, 15,  1, 10,  8,  3,  3,  2,  7,  3,  1,  3,  3,  1,  1,  0,  3,
          24,  4, 25,  4, 26,  0,

In [7]:
sample = BaseCollator(max_seq_len=50)([boosting_dataset[idx] for idx in range(10)])

In [8]:
d(sample)

ModelBatch(categorical=tensor([[[ 7, 13,  2,  ...,  0,  0,  0],
         [ 7,  8, 11,  ...,  0,  0,  0],
         [ 7,  7,  6,  ...,  0,  0,  0],
         ...,
         [ 0,  0,  0,  ...,  0,  0,  0],
         [ 0,  0,  0,  ...,  0,  0,  0],
         [ 0,  0,  0,  ...,  0,  0,  0]],

        [[17,  9, 12,  ...,  0,  0,  0],
         [ 3, 16,  6,  ...,  0,  0,  0],
         [12, 11, 11,  ...,  0,  0,  0],
         ...,
         [ 0,  0,  0,  ...,  0,  0,  0],
         [ 0,  0,  0,  ...,  0,  0,  0],
         [ 0,  0,  0,  ...,  0,  0,  0]],

        [[ 2, 11,  8,  ...,  0,  0,  0],
         [ 2, 11, 13,  ..., 18,  0,  0],
         [ 2,  7,  6,  ..., 11,  0,  0],
         ...,
         [ 0,  0,  0,  ...,  0,  0,  0],
         [ 0,  0,  0,  ...,  0,  0,  0],
         [ 0,  0,  0,  ...,  0,  0,  0]],

        ...,

        [[14, 12, 15,  ...,  0,  0,  0],
         [ 8,  6, 16,  ...,  0,  0,  0],
         [12,  0, 13,  ...,  0,  0,  0],
         ...,
         [ 0,  0,  0,  ...,  0,  0,  0],

In [9]:
d(sample.targets.size())

torch.Size([10, 1])

In [10]:
d(ModelBatch(
    categorical=torch.zeros(size=(1, 50, 31), dtype=torch.float),
    targets=torch.zeros(size=(1, 1), dtype=torch.float),
    sample_indexes=[" " for _ in range(1)],
    mask=torch.zeros(size=(1, ), dtype=torch.bool)
))

ModelBatch(categorical=tensor([[[0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         ...,
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.]]]), targets=tensor([[0.]]), sample_indexes=[' '], mask=tensor([False]))

In [11]:
class PredictLitModel:
    def __init__(self, ckpt_path: str):
        trained_model = TabularRNNLitModule.load_from_checkpoint(ckpt_path)
        trained_model.eval()
        trained_model.freeze()

        self.net = trained_model

    def predict(self, sample):
        return self.net(sample)

In [12]:
model = PredictLitModel(
    "/home/danil/projects/torch_template/nfs/samples_RNN/LHT_credits-history/logs/train/runs/2024-03-30_23-16-50/checkpoints/epoch_003.ckpt"
)

In [13]:
d(model.net)

TabularRNNLitModule(
  (net): SequentialLitModel(
    (layers): Sequential(
      (0): EncoderLayer(
        (dropout): Dropout(p=0.3, inplace=False)
        (embeddings): EmbeddingLayer(
          (embeddings): ModuleList(
            (0-1): 2 x Embedding(18, 32)
            (2-3): 2 x Embedding(17, 32)
            (4): Embedding(16, 32)
            (5): Embedding(20, 32)
            (6): Embedding(7, 32)
            (7): Embedding(6, 32)
            (8): Embedding(4, 32)
            (9): Embedding(14, 32)
            (10): Embedding(4, 32)
            (11-12): 2 x Embedding(7, 32)
            (13): Embedding(6, 32)
            (14): Embedding(4, 32)
            (15-16): 2 x Embedding(2, 32)
            (17): Embedding(4, 32)
            (18): Embedding(25, 32)
            (19): Embedding(5, 32)
            (20): Embedding(26, 32)
            (21): Embedding(5, 32)
            (22): Embedding(27, 32)
            (23): Embedding(7, 32)
            (24): Embedding(19, 32)
            (2

In [15]:
d(model.predict(sample))

ModelOutput(preds=tensor([[-0.1699],
        [ 0.6547],
        [ 1.5091],
        [ 1.2857],
        [-0.1645],
        [-0.1696],
        [ 1.4894],
        [-0.1699],
        [-0.1370],
        [ 1.2811]]))

In [ ]:
sample = DiskFile(
    data_path="./data/credits-history/serialized/ser_full_0/data.txt", 
    indexes_path="./data/credits-history/serialized/ser_full_0/index.txt"
)

In [ ]:
indexes = np.asarray(list(sample.indexes.keys()), dtype=str)

In [20]:
d(max([sample[idx]["length"] for idx in indexes]))

51

## make indexes

In [107]:
d(int(0.99 * len(indexes)))

247500

In [103]:
train_indexes = np.random.choice(indexes, size=int(0.99 * len(indexes)), replace=False)

In [104]:
valid_indexes = indexes[~np.isin(indexes, train_indexes)]

In [108]:
with open("./data/credits-history/indexes/ser_full_0_indexes/train_indexes.pickle", "wb") as output:
    pickle.dump(train_indexes.astype(str).tolist(), output)

In [109]:
with open("./data/credits-history/indexes/ser_full_0_indexes/valid_indexes.pickle", "wb") as output:
    pickle.dump(valid_indexes.astype(str).tolist(), output)

In [9]:
new_indexes = {str(key): item for key, item in sample.indexes.items()}

In [11]:
with open("./data/credits-history/serialized/index_new.txt", "wb") as output:
    output.write(serialize_and_compress(new_indexes))

In [9]:
d(sample.indexes.__len__())

100000

In [23]:
!du -h ./data/credits-history/serialized/ser_100k_first/

109M	./data/credits-history/serialized/ser_100k_first/
